In [1]:
CUDA_LAUNCH_BLOCKING=1.

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns 
from torchvision import transforms
from PIL import Image
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [3]:
from torchvision import datasets
data_path = '/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True, transform =  transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /p1ch7/cifar-10-python.tar.gz to /p1ch7/
Files already downloaded and verified


Problem 1 Part 1

In [4]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            #???#
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(8*8*8, 32),
            nn.Tanh(),
            nn.Linear(32, 10))

In [5]:
class Net (nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [6]:
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(8*8*8, 32)
    self.fc2 = nn.Linear(32, 10)
  
  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
    out = out.view(-1, 8*8*8)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [7]:
device = (torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda:0.


In [8]:
torch.cuda.is_available()

True

In [9]:
import datetime
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs.to('cuda:0'))
      loss = loss_fn(outputs.to('cuda:0'), labels.to('cuda:0'))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      if epoch == 1 or epoch % 50 == 0:
        print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

Streaming output truncated to the last 5000 lines.
2022-03-26 16:47:38.314182 Epoch 1, Training Loss 1.285469469664347
2022-03-26 16:47:38.340856 Epoch 1, Training Loss 1.2878747910184933
2022-03-26 16:47:38.365614 Epoch 1, Training Loss 1.2903980629523393
2022-03-26 16:47:38.396854 Epoch 1, Training Loss 1.2928680350713413
2022-03-26 16:47:38.421509 Epoch 1, Training Loss 1.2955021685956385
2022-03-26 16:47:38.447661 Epoch 1, Training Loss 1.2981444659745296
2022-03-26 16:47:38.471285 Epoch 1, Training Loss 1.3006800179896147
2022-03-26 16:47:38.495678 Epoch 1, Training Loss 1.3032757685617413
2022-03-26 16:47:38.520349 Epoch 1, Training Loss 1.3057935788198505
2022-03-26 16:47:38.544043 Epoch 1, Training Loss 1.3082712557919496
2022-03-26 16:47:38.572927 Epoch 1, Training Loss 1.3108228306331293
2022-03-26 16:47:38.597812 Epoch 1, Training Loss 1.3133904978137492
2022-03-26 16:47:38.628945 Epoch 1, Training Loss 1.3158069612729886
2022-03-26 16:47:38.653129 Epoch 1, Training Loss 1.3

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
      correct = 0
      total = 0
    
      with torch.no_grad():
        for imgs, labels in loader:
          outputs = model(imgs.to('cuda:0'))
          _, predicted = torch.max(outputs.to('cuda:0'), dim=1)
          total += labels.shape[0]
          correct += int((predicted.to('cuda:0') == labels.to('cuda:0')).sum())
    
      print("Accuracy {}: {:.2f}".format(name, correct/total))
  

In [31]:
validate(model, train_loader, val_loader)

Accuracy train: 0.83
Accuracy val: 0.62


Problem 1 Part 2

In [32]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(8,4, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(4*4*4, 32),
            nn.Tanh(),
            nn.Linear(32, 10))

In [33]:
device = (torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda:0.


In [34]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

Streaming output truncated to the last 5000 lines.
2022-03-26 18:56:01.101308 Epoch 1, Training Loss 1.2934900006979628
2022-03-26 18:56:01.130706 Epoch 1, Training Loss 1.2960777843699736
2022-03-26 18:56:01.154474 Epoch 1, Training Loss 1.2986304610586532
2022-03-26 18:56:01.177788 Epoch 1, Training Loss 1.3013318987453686
2022-03-26 18:56:01.205197 Epoch 1, Training Loss 1.303989147896047
2022-03-26 18:56:01.241344 Epoch 1, Training Loss 1.3064948196911141
2022-03-26 18:56:01.265036 Epoch 1, Training Loss 1.3090027355781906
2022-03-26 18:56:01.289266 Epoch 1, Training Loss 1.3115314899198234
2022-03-26 18:56:01.312878 Epoch 1, Training Loss 1.3140480091504734
2022-03-26 18:56:01.344243 Epoch 1, Training Loss 1.316501781458745
2022-03-26 18:56:01.368998 Epoch 1, Training Loss 1.3191832560102652
2022-03-26 18:56:01.393291 Epoch 1, Training Loss 1.3217977102455276
2022-03-26 18:56:01.427263 Epoch 1, Training Loss 1.3242343548313735
2022-03-26 18:56:01.457348 Epoch 1, Training Loss 1.32

In [35]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.82
Accuracy val: 0.62


Problem 2 Part 1

In [11]:
class NetDepth(nn.Module): 
  def __init__(self, n_chans1=32):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding =1)
    self.fc1 = nn.Linear(4*4*n_chans1 // 2, 32)
    self.fc2 = nn.Linear(32, 2)
  
  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
    out = F.max_pool2d(torch.relu(self.conv3(out)), 2)
    out = out.view(-1, 4*4*self.n_chans1 // 2)
    out = self.fc2(out)
    return out

In [12]:
class NetRes(nn.Module):
  def __init__(self, n_chans1=32):
    super().__init__()
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(4*4*n_chans1 // 2, 32)
    self.fc2 = nn.Linear(32, 2)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
    out1 = out
    out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
    out = out.view(-1, 4*4*self.n_chans1 // 2)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out


In [13]:
class ResBlock(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    #self.btach_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    #torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    #torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = torch.relu(out)
    return out + x

In [14]:
class ResNet10(nn.Module): 
  def __init__(self, n_chans1=32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Sequential(
        *(n_blocks * [ResBlock(n_chans=n_chans1)]))
    self.fc1 = nn.Linear(8*8*n_chans1, 32)
    self.fc2 = nn.Linear(32,2)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8*8*self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

In [15]:
model = ResNet10()
model.to('cuda:0')

ResNet10(
  (conv1): Sequential(
    (0): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (3): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (4): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (5): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (6): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (7): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (8): ResBlock(
      (conv)

In [17]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-03-26 20:33:13.285253 Epoch 1, Training Loss 0.002920018437573367
2022-03-26 20:33:13.316260 Epoch 1, Training Loss 0.005855738964227155
2022-03-26 20:33:13.346358 Epoch 1, Training Loss 0.008834523008302654
2022-03-26 20:33:13.376524 Epoch 1, Training Loss 0.011784244681258335
2022-03-26 20:33:13.407699 Epoch 1, Training Loss 0.014759219820846986
2022-03-26 20:33:13.437814 Epoch 1, Training Loss 0.01771200252006121
2022-03-26 20:33:13.468104 Epoch 1, Training Loss 0.02066094796066089
2022-03-26 20:33:13.498466 Epoch 1, Training Loss 0.02363117942419808
2022-03-26 20:33:13.529480 Epoch 1, Training Loss 0.026609259493210736
2022-03-26 20:33:13.559498 Epoch 1, Training Loss 0.029566505985796604
2022-03-26 20:33:13.585252 Epoch 1, Training Loss 0.032516887120883485
2022-03-26 20:33:13.611608 Epoch 1, Training Loss 0.0354517271451633
2022-03-26 20:33:13.638626 Epoch 1, Training Loss 0.03842124640179412
2022-03-26 20:33:13.665632 Epoch 1, Training Loss 0.04138691681425285
2022-03-26 20

In [18]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.81
Accuracy val: 0.61


Problem 2 Part 2 Subsection A: Using Batch Normilization

In [21]:
class ResBlock(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = torch.relu(out)
    return out + x

In [23]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-03-26 21:47:20.061424 Epoch 1, Training Loss 0.0029193165966921756
2022-03-26 21:47:20.091432 Epoch 1, Training Loss 0.005855395970746989
2022-03-26 21:47:20.122439 Epoch 1, Training Loss 0.008823782586685532
2022-03-26 21:47:20.152445 Epoch 1, Training Loss 0.0117649425326101
2022-03-26 21:47:20.184452 Epoch 1, Training Loss 0.014716754788937777
2022-03-26 21:47:20.215459 Epoch 1, Training Loss 0.01767643729744055
2022-03-26 21:47:20.246468 Epoch 1, Training Loss 0.020637740564468265
2022-03-26 21:47:20.277476 Epoch 1, Training Loss 0.02355118755184476
2022-03-26 21:47:20.304499 Epoch 1, Training Loss 0.026497203980565376
2022-03-26 21:47:20.330506 Epoch 1, Training Loss 0.02941852396406481
2022-03-26 21:47:20.357511 Epoch 1, Training Loss 0.032361017468640264
2022-03-26 21:47:20.384499 Epoch 1, Training Loss 0.03529807978578846
2022-03-26 21:47:20.411524 Epoch 1, Training Loss 0.038255017431800625
2022-03-26 21:47:20.437524 Epoch 1, Training Loss 0.041236679877161675
2022-03-26 

In [24]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.82
Accuracy val: 0.61


Problem 2 Part 2 Subsection  B: Using Dropout with p=0.3

---



In [25]:
class ResNet10(nn.Module):
  def __init__(self, n_chans1=32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Sequential(
        *(n_blocks * [ResBlock(n_chans=n_chans1)]))
    self.conv1_dropout = nn.Dropout2d(p=0.3)
    self.fc1 = nn.Linear(8*8*n_chans1 // 2, 32)
    self.fc2 = nn.Linear(32,2)

  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = self.conv1_dropout(out)
    out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
    out = self.conv2_dropout(out)
    out = out.view(-1, 8*8*self.n_chans1 // 2)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [26]:
model = ResNet10()
model.to('cuda:0')

ResNet10(
  (conv1): Sequential(
    (0): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [27]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-03-26 22:56:24.232976 Epoch 1, Training Loss 0.0029549324299063522
2022-03-26 22:56:24.253956 Epoch 1, Training Loss 0.005928043819144559
2022-03-26 22:56:24.273961 Epoch 1, Training Loss 0.008855945618866046
2022-03-26 22:56:24.292974 Epoch 1, Training Loss 0.011795571393064221
2022-03-26 22:56:24.311983 Epoch 1, Training Loss 0.014728807122506144
2022-03-26 22:56:24.327724 Epoch 1, Training Loss 0.017657711377838992
2022-03-26 22:56:24.345704 Epoch 1, Training Loss 0.02061652496952535
2022-03-26 22:56:24.370150 Epoch 1, Training Loss 0.023545584105469686
2022-03-26 22:56:24.389968 Epoch 1, Training Loss 0.02649913937844279
2022-03-26 22:56:24.406972 Epoch 1, Training Loss 0.029452876056856512
2022-03-26 22:56:24.424994 Epoch 1, Training Loss 0.03237599820432151
2022-03-26 22:56:24.441188 Epoch 1, Training Loss 0.03529697580410696
2022-03-26 22:56:24.457202 Epoch 1, Training Loss 0.038219732396742874
2022-03-26 22:56:24.475289 Epoch 1, Training Loss 0.041148569272912064
2022-03-2

In [28]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.81
Accuracy val: 0.62


Problem 2 Part 2 Subsection C: Weight Decay with lambda of 0.001

In [30]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs.to('cuda:0'))
      loss = loss_fn(outputs.to('cuda:0'), labels.to('cuda:0'))
      
      ambda = 0.001
      norm = sum(p.pow(2.0).sum()
                    for p in model.parameters())
    
      loss = loss + ambda*norm

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      if epoch == 1 or epoch % 50 == 0:
        print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [31]:
model = ResNet10()
model.to('cuda:0')

ResNet10(
  (conv1): Sequential(
    (0): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [32]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-03-27 00:13:00.326515 Epoch 1, Training Loss 0.002978465441242813
2022-03-27 00:13:00.358614 Epoch 1, Training Loss 0.005968467353859826
2022-03-27 00:13:00.388956 Epoch 1, Training Loss 0.008944835504302587
2022-03-27 00:13:00.420205 Epoch 1, Training Loss 0.011918019455717042
2022-03-27 00:13:00.452213 Epoch 1, Training Loss 0.014888463422770391
2022-03-27 00:13:00.483219 Epoch 1, Training Loss 0.017841788509007916
2022-03-27 00:13:00.512240 Epoch 1, Training Loss 0.020803628370280155
2022-03-27 00:13:00.539246 Epoch 1, Training Loss 0.02381051928186051
2022-03-27 00:13:00.567246 Epoch 1, Training Loss 0.026784718189093157
2022-03-27 00:13:00.594259 Epoch 1, Training Loss 0.029749081262847043
2022-03-27 00:13:00.622259 Epoch 1, Training Loss 0.032688901552458856
2022-03-27 00:13:00.649265 Epoch 1, Training Loss 0.035659949797803486
2022-03-27 00:13:00.676395 Epoch 1, Training Loss 0.038625204654605796
2022-03-27 00:13:00.702764 Epoch 1, Training Loss 0.04159971301817833
2022-03-

In [33]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.76
Accuracy val: 0.68
